In [33]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re
from utils import TJData, setup_env, APIHandler, nan_to_int
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api.models import Comarca, Serventia, Assunto, Classe, ClasseAssunto, Competencia, Processo, ProcessoUnico, TipoPersonagem
# objeto para interagir com a minha API
api = APIHandler('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

In [ ]:
comarcas = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','DESC_REDU'])
# filtra as linhas com cod_coma nulas
comarcas = comarcas[~pd.isnull(comarcas.COD_COMA)]
# converte cod_coma  para inteiro
comarcas.COD_COMA = comarcas.COD_COMA.apply(lambda x: int(x))
# converte o cabeçalho para caixa baixa
comarcas = comarcas.rename(columns=lambda x: str(x).lower())
# gera lista de objetos json a partir do dataframe
comarca_json = json.loads(comarcas.to_json(orient='records'))
# executa chamada a api
r = api.post('comarcas',comarca_json)

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

In [ ]:
serventias = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE'] )
serventias = serventias[~pd.isnull(serventias.COD_COMA)]
serventias.COD_COMA = serventias.COD_COMA.apply(lambda x: int(x))
# elimina codigos não referenciados
serventias = serventias[serventias.COD_COMA != 888]
serventias = serventias[serventias.COD_COMA != 999]
serventias = serventias.rename(columns=lambda x: str(x).lower())
# ajusta cod_coma para referenciar o atributo comarca(FK) de serventia
serventias = serventias.rename(columns={'cod_coma':'comarca'})
serventia_json = json.loads(serventias.to_json(orient='records'))
r = api.post('serventias',serventia_json)

### Competencias

In [ ]:
competencias = TJData.read_csv(TJData.csv['competencia'], usecols=['COD_COMP', 'DESC_COMP', 'DESC_RES'])
competencias = competencias.rename(columns=lambda x: str(x).lower())
competencia_json = json.loads(competencias.to_json(orient='records'))
r = api.post('competencias',competencia_json)

### Assunto
tabela de assuntos é incompleta, é necessário filtrar

In [111]:
def load_assunto(error_set=[]):
    assunto = TJData.read_csv(TJData.csv['assunto'], usecols=['COD_ASSUNTO', 'COD_ASSUNTO_PAI', 'DESCR'])
    assunto = assunto.rename(columns=lambda x: str(x).lower())
    assunto = assunto.rename(columns={'cod_assunto_pai':'assunto_pai'})
    assunto = assunto.sort_values(by='cod_assunto', ascending=True)
    # converte COD_ASSUNTO_PAI para inteiro
    assunto.assunto_pai = nan_to_int(assunto,'assunto_pai')

    # filtra COD_ASSUNTO_PAI que não possuem referencia
    assunto = assunto[~assunto.assunto_pai.isin(error_set)]

    assunto_json = json.loads(assunto.to_json(orient='records'))
    r = api.post('assuntos', assunto_json)
    return r

In [108]:
def handle_assunto_pai_error(r):
    # processa cod_assunto_pai que ocorreu erro
    r_error = json.loads(r.text)
    r_error = pd.DataFrame(r_error)
    r_error.assunto_pai = r_error.assunto_pai.astype(str)
    r_error.assunto_pai = r_error.assunto_pai.str.extract(r'([0-9]+)')
    r_error = r_error[~pd.isna(r_error.assunto_pai)]
    r_error.assunto_pai = r_error.assunto_pai.astype(int)
    error_set = set()
    r_error.assunto_pai.apply(lambda x : error_set.add(x))
    error_set = list(error_set)
    return error_set

In [112]:
r = load_assunto()

In [115]:
error_set = handle_assunto_pai_error(r)

In [117]:
r2 = load_assunto(error_set=error_set)

In [123]:
assunto = TJData.read_csv(TJData.csv['assunto'], usecols=['COD_ASSUNTO', 'COD_ASSUNTO_PAI', 'DESCR'])
assunto = assunto.rename(columns=lambda x: str(x).lower())
assunto = assunto.rename(columns={'cod_assunto_pai':'assunto_pai'})
assunto = assunto.sort_values(by='cod_assunto', ascending=True)
# converte COD_ASSUNTO_PAI para inteiro
assunto.assunto_pai = nan_to_int(assunto,'assunto_pai')
assunto

,cod_assunto,assunto_pai,descr
857,14,NaN,DIREITO TRIBUTÁRIO
858,195,NaN,DIREITO PREVIDENCIÁRIO
859,287,NaN,DIREITO PENAL
860,864,NaN,DIREITO DO TRABALHO
861,899,NaN,DIREITO CIVIL
862,1146,NaN,DIREITO MARÍTIMO
863,1156,NaN,DIREITO DO CONSUMIDOR
864,1209,NaN,DIREITO PROCESSUAL PENAL
865,1654,864,Contrato Individual de Trabalho
866,1658,864,Duração do Trabalho


In [142]:
assunto_json = json.loads(assunto.to_json(orient='records'))
r = api.post('assuntos', assunto_json)

In [143]:
r.text

'{"assunto_pai":["Invalid pk \\"2594\\" - object does not exist."]}'

In [141]:
Assunto.objects.all()

<QuerySet []>